In [3]:
from ClassFiles.networks import SebastianConvNet
NN = SebastianConvNet(1, 128, 128)
NN.to("cuda")

SebastianConvNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (conv4): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (conv5): Conv2d(64, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (conv6): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (linear1): Linear(in_features=8192, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=1, bias=True)
)

In [6]:
from algorithm1 import train
NN = train(NN, 'local_testing_net', epochs = 10, batch_size = 50 , mu = 1)

untrained performance 0.01816600374877453 0.01816578023135662
done epoch -8.000093460083008 16.019254684448242
done epoch -195.02183532714844 -169.8485565185547
done epoch -126.47249603271484 -99.04129028320312
done epoch -161.04489135742188 -128.82681274414062
done epoch -278.68670654296875 -246.34097290039062
done epoch -222.69346618652344 -188.8977508544922
done epoch -284.9851989746094 -258.94720458984375
done epoch -188.89401245117188 -157.72146606445312
done epoch -173.2315216064453 -143.59625244140625
done epoch -300.61090087890625 -278.0100402832031


In [9]:
import torch
from ClassFiles.DataLoader import get_generated_dataloader
from ClassFiles.networks import SebastianConvNet
from algorithm2_old import reconstruct, minimum

eval_groundtruth_loader = get_generated_dataloader('eval', 'clean', batch_size = 100, shuffle = False)
eval_chanvese_loader = get_generated_dataloader('eval', 'chan-vese', batch_size = 100, shuffle = False)
eval_noisy_loader = get_generated_dataloader('eval', 'dirty', batch_size = 100, shuffle = False)

eval_groundtruth_batch = iter(eval_groundtruth_loader).next()[0].to("cuda")
eval_chanvese_batch = iter(eval_chanvese_loader).next()[0].to("cuda")
eval_noisy_batch = iter(eval_noisy_loader).next()[0].to("cuda")

NN.to("cuda")

#reconstructed_batch, quality, steps = minimum(eval_chanvese_batch, eval_noisy_batch, eval_groundtruth_batch, NN, 1, 0.001)
#print(steps)

reconstructed_batch, _ = reconstruct(eval_chanvese_batch, eval_noisy_batch, NN, 1, 0.004, 50) #c1 = torch.zeros([1], device = "cuda"), c2 = torch.ones([1], device = "cuda"))

#reconstructed_batch, _ = reconstruct(eval_noisy_batch, eval_noisy_batch, NN, 0, 16000, 400) #c1 = torch.zeros([1], device = "cuda"), c2 = torch.ones([1], device = "cuda"))

#reconstructed_batch, _ = shitty_reconstruct(eval_chanvese_batch, eval_noisy_batch, NN, 0.01, 50)

print(NN(reconstructed_batch).mean().item(), NN(eval_groundtruth_batch).mean().item(), NN(eval_chanvese_batch).mean().item()) 

reconstructed_batch_numpy = reconstructed_batch.to("cpu").numpy()
reconstructed_batch_numpy_squeeze = reconstructed_batch_numpy.squeeze()

from PIL import Image
for i in range(100):
    im = Image.fromarray(255*reconstructed_batch_numpy_squeeze[i]).convert("L")
    im.save(fp =  "eval/images/reconstructed/reconstructed_"+str(i).zfill(2)+".png", format = 'PNG')

-1111.72314453125 -300.61090087890625 -278.0100402832031
